<a href="https://colab.research.google.com/github/fkatelyn/congressional2023/blob/main/notebooks/palm_tree_gradio_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
MODEL_VERSION = '7'

In [3]:
# prompt: write a gradio which take image input and image output. Use yolov8 from analytics to do the input image processing. The processed image will be the output.

!pip install ultralytics==8.0.134
!pip install gradio






     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.1/629.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 5.4 MB/s eta 0

In [5]:
import os
HOME = os.getcwd()
HOME = '/content'
print(HOME)

from google.colab import drive
GOOGLE_DRIVE = f'{HOME}/drive'
drive.mount(GOOGLE_DRIVE)

/content
Mounted at /content/drive


In [6]:
class Experiment:
  '''
  Google drive path:

  PROJECT_DRIVE = MyDrive/ai/projects/palmtree
  EXPERIMENT_DRIVE = MyDrive/ai/projects/palmtree/experiments
  UPLOAD_DRIVE = MyDrive/ai/projects/palmtree/uploads
  DATASET_DRIVE = MyDrive/ai/projects/palmtree/datasets
  '''
  EPOCHS = 200
  IMGSZ = 640
  CONFIDENT = 0.15
  MODEL_NAME = "yolov8s"
  EXPERIMENT_HOME = f'{HOME}/experiments/{MODEL_VERSION}'
  DATASET_HOME = f'{HOME}/datasets/palm-tree-detection-{MODEL_VERSION}'
  WEIGHTS_HOME = f'{HOME}/weights/{MODEL_VERSION}'

  PROJECT_DRIVE = f'{GOOGLE_DRIVE}/MyDrive/ai/projects/palmtree'
  EXPERIMENT_DRIVE = f'{PROJECT_DRIVE}/experiments/{MODEL_VERSION}'
  DATASET_DRIVE = f'{PROJECT_DRIVE}/datasets/palm-tree-detection-{MODEL_VERSION}'
  WEIGHTS_DRIVE = f'{PROJECT_DRIVE}/weights/{MODEL_VERSION}'


print(Experiment.EXPERIMENT_HOME)


/content/experiments/7


In [7]:

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.134 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.4/225.8 GB disk)


In [8]:
%cd {HOME}
%mkdir -p {HOME}/weights/
!cp -fpR {Experiment.WEIGHTS_DRIVE} /content/weights/


/content


In [10]:
from PIL import Image
import gradio as gr
import torch
from ultralytics import YOLO

model = YOLO(f'{Experiment.WEIGHTS_HOME}/best.pt')

def yolo(im, size=Experiment.IMGSZ):
    g = (size / max(im.size))  # gain
    im = im.resize((int(x * g) for x in im.size), Image.LANCZOS)

    results = model.predict(im, conf=Experiment.CONFIDENT)
    im_array = results[0].plot()  # updates results.imgs with boxes and labels
    imo = Image.fromarray(im_array[..., ::-1])  # RGB

    return imo


inputs = gr.inputs.Image(type='pil', label="Original Image")
outputs = gr.outputs.Image(type="pil", label="Output Image")

title = "Palm Oil Tree detection"
description = "Upload an image or click an example image to use."
examples = ['test1.jpg']

gr.Interface(yolo,
             inputs,
             outputs,
             title=title,
             description=description,
             examples=examples,
             analytics_enabled=False).launch(debug=True)

<ipython-input-10-fa60a99c3e6f>:19: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs = gr.inputs.Image(type='pil', label="Original Image")
<ipython-input-10-fa60a99c3e6f>:19: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs = gr.inputs.Image(type='pil', label="Original Image")
<ipython-input-10-fa60a99c3e6f>:20: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs = gr.outputs.Image(type="pil", label="Output Image")


FileNotFoundError: ignored